# Feature Importances

## Comparing ALE estimates and SHAP values

In [ ]:
import math
from concurrent.futures import ProcessPoolExecutor, wait
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shap import (
    TreeExplainer,
    decision_plot,
    dependence_plot,
    force_plot,
    initjs,
    summary_plot,
)
from sklearn.ensemble import RandomForestRegressor
from tqdm.auto import tqdm

from alepython import ale_plot

initjs()

In [ ]:
np.random.seed(1)
X = pd.DataFrame(np.random.random((int(1e3), 2)), columns=["a", "b"])

# Construct the output variable.
y = 0.5 * X["a"] - 1 * X["b"] + 0.02 * np.random.random(X.shape[0])

# Introduce an interaction effect between 'a' and 'b'.
y[X["b"] > 0.5] -= X["a"]

model = RandomForestRegressor(
    n_estimators=100, max_depth=10, min_samples_leaf=4, random_state=1, n_jobs=-1,
)
model.fit(X, y)

### Get first-order ALE plots and data

In [ ]:
first_order_data = []
for feature in tqdm(X.columns, desc="Feature ALEs"):
    fig, axes, data = ale_plot(
        model,
        X,
        feature,
        bins=20,
        fig=plt.figure(),
        monte_carlo=True,
        monte_carlo_rep=200,
        monte_carlo_ratio=20,
        verbose=True,
        plot_quantiles=True,
        center=True,
        quantile_axis=True,
        return_data=True,
    )
    first_order_data.append(data)
    axes["ale"].xaxis.set_tick_params(rotation=45)
    axes["quantiles_x"].xaxis.set_tick_params(rotation=45)

In [ ]:
first_order_data = []
for feature in tqdm(X.columns, desc="Feature ALEs"):
    fig, axes, data = ale_plot(
        model,
        X,
        feature,
        bins=20,
        fig=plt.figure(),
        monte_carlo=True,
        monte_carlo_rep=200,
        monte_carlo_ratio=20,
        verbose=True,
        plot_quantiles=True,
        center=False,
        quantile_axis=True,
        return_data=True,
    )
    first_order_data.append(data)
    axes["ale"].xaxis.set_tick_params(rotation=45)
    axes["quantiles_x"].xaxis.set_tick_params(rotation=45)

### Get second-order ALE plots and data

In [ ]:
second_order_data = []
for features in tqdm(list(combinations(X.columns, 2)), desc="Feature pair 2D ALEs"):
    fig, axes, data = ale_plot(
        model,
        X,
        features,
        bins=20,
        fig=plt.figure(),
        plot_quantiles=True,
        quantile_axis=True,
        return_data=True,
        n_jobs=-1,
    )
    second_order_data.append(data)
    axes["ale"].xaxis.set_tick_params(rotation=45)
    axes["quantiles_x"].xaxis.set_tick_params(rotation=45)

### Importances based off the vertical extent of the first-order ALE plots

In [ ]:
first_order_imps = {}
for feature, data in zip(X.columns, first_order_data):
    quantiles, ale = data
    first_order_imps[feature] = np.ptp(ale)
first_order_imps = pd.Series(first_order_imps, name="1st Order Importance")
first_order_imps

### Importances based off the amplitude of the second-order ALE plots

In [ ]:
second_order_imps = {}
for features, data in zip(combinations(X.columns, 2), second_order_data):
    quantiles, ale, samples = data
    second_order_imps[features] = np.ptp(ale)
second_order_imps = pd.Series(second_order_imps, name="2nd Order Importance")
second_order_imps

### Evaluate the feature correlations

In [ ]:
X.corr()

### SHAP values

In [ ]:
explainer = TreeExplainer(model)

In [ ]:
N = X.shape[0]
chunks = 10
chunksize = math.ceil(N / chunks)

In [ ]:
%%time


def get_shap_values(index):
    return explainer.shap_values(X[index * chunksize : (index + 1) * chunksize])


with ProcessPoolExecutor(max_workers=None) as executor:
    shap_values = np.vstack(list(executor.map(get_shap_values, range(chunks))))

In [ ]:
%%time


def get_shap_interaction_values(index):
    return explainer.shap_interaction_values(
        X[index * chunksize : (index + 1) * chunksize]
    )


with ProcessPoolExecutor(max_workers=None) as executor:
    shap_interaction_values = np.vstack(
        list(executor.map(get_shap_interaction_values, range(chunks)))
    )

In [ ]:
summary_plot(shap_values, X, alpha=0.5)

In [ ]:
summary_plot(shap_interaction_values, X)

In [ ]:
summary_plot(shap_interaction_values, X, plot_type="compact_dot")

In [ ]:
dependence_plot("a", shap_values, X)

In [ ]:
dependence_plot("b", shap_values, X)

In [ ]:
force_plot(np.mean(model.predict(X)), shap_values, X)

In [ ]:
shap_values.shape

In [ ]:
shap_interaction_values.shape

In [ ]:
plt.hist(shap_values[:, 0])

In [ ]:
plt.hist(
    shap_interaction_values[:, 0, 0]
    + shap_interaction_values[:, 0, 1]
    - shap_values[:, 0]
)

In [ ]:
plt.hist(
    shap_interaction_values[:, 0, 0]
    + shap_interaction_values[:, 1, 0]
    - shap_values[:, 0]
)

In [ ]:
plt.hist(
    shap_interaction_values[:, 0, 0]
    + 2 * shap_interaction_values[:, 1, 0]
    - shap_values[:, 0]
)

In [ ]:
plt.hist(shap_interaction_values[:, 1, 0] - shap_interaction_values[:, 0, 1])